In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from sklearn.utils import shuffle
from PIL import ImageFont
import visualkeras

In [5]:
model = load_model('/home/blizzard/Datasets/Models/s4')

In [6]:
files = ['ttbar','wmp','wpwm','zwpm','n2n2']

cs_lo_k = {
            'ttbar':988.57,
            'wmp'  :1.95*1e5,
            'wpwm' :124.31,
            'zwpm' :51.82,
            'n2n2' :1
          }

br_ratio = {
            'ttbar':0.67**2,
            'wmp'  :(1-0.67),
            'wpwm' :(1-0.67)*0.67,
            'zwpm' :0.7*(1-0.67),
            'n2n2' :1
          }

cs_nmg = {
         'ttbar':393.30,
         'wmp'  :7.865*1e4,
         'wpwm' :74.96,
         'zwpm' :14.28,
         'n2n2' :1
         }

cs_mg = {'ttbar':2.558,
          'wmp':36.8,
          'wpwm':2.9*1e-1,
          'zwpm':7.33*1e-2,
          'n2n2':3.99*1e-4
        }

cs_pb = []
for f in files:
  cs_pb.append((cs_lo_k[f]*br_ratio[f]*cs_mg[f])/cs_nmg[f])

cs = [i*1e3 for i in cs_pb]
#k_f = [1.954,1.356,1.92,2.09,1.0]

cs_corr = {files[i] : cs[i] for i in range(len(files))}

In [7]:
df = []
for f in range(len(files)):
    con_df = []
    
    for i in range(11,12):
        con_df.append(pd.read_csv('~/Datasets/Comp/' + files[f] + str(i) + '.csv'))
    
    df.append(pd.concat(con_df,ignore_index=True))
    df[-1]['type'] = f
    
    if files[f] == "n2n2":
        df[-1]['tag'] = 1
    else:
        df[-1]['tag'] = 0

In [8]:
dtset = pd.concat(df,ignore_index=True)
dtset = shuffle(dtset)
dtset['met'] = np.fabs(dtset['met'])

In [9]:
## Analysis Level Cuts
dtset = dtset[dtset['ptl'] >= 120.0][dtset['ptj'] >= 120.0][dtset['etaj'] <= 2.0][dtset['etaj'] >= -2.0]

/tmp/ipykernel_23803/3308945723.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dtset = dtset[dtset['ptl'] >= 120.0][dtset['ptj'] >= 120.0][dtset['etaj'] <= 2.0][dtset['etaj'] >= -2.0]


In [12]:
with tf.device('/device:GPU:0'):
    tot_pred = model.predict(dtset.T[:-2].T)

In [13]:
pred_set = dtset.copy()
pred_set['pred'] = tot_pred

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
print('Correctly identified signal (True Positive)     : ',len(pred_set[pred_set['pred'] >= 0.5][pred_set['tag'] == 1]))
print('Falsely identified signal (Flase Positive)      : ',len(pred_set[pred_set['pred'] >= 0.5][pred_set['tag'] == 0]))
print('Correctly identified background (True Negative) : ',len(pred_set[pred_set['pred'] < 0.5][pred_set['tag'] == 0]))
print('Falsely identified background (False Negative)  : ',len(pred_set[pred_set['pred'] < 0.5][pred_set['tag'] == 1]))

Correctly identified signal (True Positive)     :  38068
Falsely identified signal (Flase Positive)      :  137
Correctly identified background (True Negative) :  159269
Falsely identified background (False Negative)  :  43058


In [16]:
print('The amount of signal left is     :', len(pred_set[pred_set['pred'] >= 0.5][pred_set['tag'] == 1])/len(df[-1]))
print('The amount of background left is :', len(pred_set[pred_set['pred'] >= 0.5][pred_set['tag'] == 0])/np.sum([len(i) for i in df[:-1]]))

The amount of signal left is     : 0.44990190748575887
The amount of background left is : 0.0007117326794397572


In [17]:
print('Thus, the rate of correct signal prediction is : ',len(pred_set[pred_set['pred'] >= 0.5][dtset['tag'] == 1])/(len(pred_set[pred_set['pred'] >= 0.5])))

Thus, the rate of correct signal prediction is :  0.9964140819264494


In [18]:
L = 3000

In [19]:
ns = cs_corr['n2n2']*(len(pred_set[pred_set['pred'] >= 0.5][pred_set['tag'] == 1])/(1e6))*L
print('n2n2',cs_corr['n2n2'],(len(pred_set[pred_set['pred'] >= 0.5][pred_set['tag'] == 1])))
nb = 0

for i in range(len(files)-1):
    nb += cs_corr[files[i]]*(len(pred_set[pred_set['pred'] >= 0.5][pred_set['tag'] == 0][pred_set['type'] == i])/((1e6)))*L
    print(files[i],len(pred_set[pred_set['pred'] >= 0.5][pred_set['tag'] == 0][pred_set['type'] == i]),cs_corr[files[i]])

n2n2 0.399 38068
ttbar 10 2886.247873719807
wmp 9 30109.0909090909
wpwm 8 106.33181550160086
zwpm 110 61.44480294117646


In [20]:
print('The number of signal is :', ns)
print('The number of background is :', nb)
print('The significance is :',ns/np.sqrt(nb))

The number of signal is : 45.567395999999995
The number of background is : 922.3616392996751
The significance is : 1.500388044075569
